In [ ]:
import basedosdados as bd
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from plotnine import *

In [ ]:
query = """
SELECT  DISTINCT id_municipio, nome FROM `basedosdados.br_denatran_frota.municipio_tipo` 
INNER JOIN `basedosdados.br_bd_diretorios_brasil.municipio` USING (id_municipio, sigla_uf)
GROUP BY sigla_uf, id_municipio, ano, mes, nome

HAVING COUNT(*) > 1
ORDER BY id_municipio"""
frota = bd.read_sql(query, billing_project_id="python-371123")
frota